In [19]:
import datetime as dt
from bs4 import BeautifulSoup
import requests
import re
import time
import random
import pandas as pd
import os
import numpy as np

In [20]:
master_df = pd.read_csv('week_7.csv')
hist = pd.read_csv('pos_sal_avg.csv')
master_df = master_df.merge(hist, left_on=['Pos','Salary'], right_on=['Pos','DK salary'])

In [21]:
master_df.head(15)

,Pos,Name,Salary,TeamAbbrev,AvgPointsPerGame,Projected Points,DK salary,DK points
0,WR,DeAndre Hopkins,8200,ARI,21.35,21.3,8200.0,22.663636
1,QB,Russell Wilson,8000,SEA,31.88,23.8,8000.0,25.253333
2,RB,Alvin Kamara,7900,NO,30.12,23.6,7900.0,22.792308
3,WR,Davante Adams,7900,GB,21.10,21.6,7900.0,18.078261
4,RB,Ezekiel Elliott,7800,DAL,21.28,19.6,7800.0,20.421053
5,QB,Josh Allen,7700,BUF,27.79,22.2,7700.0,30.280000
6,WR,Michael Thomas,7600,NO,4.70,16.9,7600.0,15.870909
7,RB,Derrick Henry,7500,TEN,24.04,18.5,7500.0,19.100000
8,QB,Patrick Mahomes,7400,KC,28.08,22.0,7400.0,22.791429
9,WR,Calvin Ridley,7300,ATL,22.17,18.3,7300.0,17.802778


In [22]:
master_df['val_above_sal'] = master_df['Projected Points'] - master_df['DK points']

In [23]:
master_df['Points Per Dollar'] = master_df['Projected Points'] / \
    master_df['Salary']
master_df['Points Per 1K'] = master_df['Projected Points'] / 1000
master_df['Value'] = master_df['Projected Points'] / \
    (master_df['Salary']/1000)

In [24]:
master_df

,Pos,Name,Salary,TeamAbbrev,AvgPointsPerGame,Projected Points,DK salary,DK points,val_above_sal,Points Per Dollar,Points Per 1K,Value
0,WR,DeAndre Hopkins,8200,ARI,21.35,21.3,8200.0,22.663636,-1.363636,0.002598,0.0213,2.597561
1,QB,Russell Wilson,8000,SEA,31.88,23.8,8000.0,25.253333,-1.453333,0.002975,0.0238,2.975000
2,RB,Alvin Kamara,7900,NO,30.12,23.6,7900.0,22.792308,0.807692,0.002987,0.0236,2.987342
3,WR,Davante Adams,7900,GB,21.10,21.6,7900.0,18.078261,3.521739,0.002734,0.0216,2.734177
4,RB,Ezekiel Elliott,7800,DAL,21.28,19.6,7800.0,20.421053,-0.821053,0.002513,0.0196,2.512821
...,...,...,...,...,...,...,...,...,...,...,...,...
398,DST,Raiders,2400,LV,1.80,5.0,2400.0,6.831683,-1.831683,0.002083,0.0050,2.083333
399,DST,Cardinals,2300,ARI,7.00,6.4,2300.0,5.700935,0.699065,0.002783,0.0064,2.782609
400,DST,Lions,2200,DET,2.20,5.7,2200.0,5.894737,-0.194737,0.002591,0.0057,2.590909
401,DST,Texans,2200,HOU,3.33,4.8,2200.0,5.894737,-1.094737,0.002182,0.0048,2.181818


In [25]:
good_choices = master_df[(master_df['val_above_sal']>0) & (master_df['Projected Points']>11)]

In [26]:
good_choices.sort_values(by=['Pos','val_above_sal'], ascending=False)

,Pos,Name,Salary,TeamAbbrev,AvgPointsPerGame,Projected Points,DK salary,DK points,val_above_sal,Points Per Dollar,Points Per 1K,Value
42,WR,Keenan Allen,6200,LAC,16.72,17.7,6200.0,11.572075,6.127925,0.002855,0.0177,2.854839
63,WR,Tyler Boyd,5400,CIN,15.08,15.8,5400.0,10.844776,4.955224,0.002926,0.0158,2.925926
54,WR,Terry McLaurin,5800,WAS,16.17,16.9,5800.0,12.025373,4.874627,0.002914,0.0169,2.913793
56,WR,DJ Moore,5600,CAR,13.90,14.8,5600.0,11.217391,3.582609,0.002643,0.0148,2.642857
50,WR,Jamison Crowder,5900,NYJ,22.08,15.8,5900.0,12.243077,3.556923,0.002678,0.0158,2.677966
3,WR,Davante Adams,7900,GB,21.10,21.6,7900.0,18.078261,3.521739,0.002734,0.0216,2.734177
69,WR,Brandin Cooks,5200,HOU,13.12,14.1,5200.0,10.608732,3.491268,0.002712,0.0141,2.711538
14,WR,Stefon Diggs,7000,BUF,20.75,18.6,7000.0,15.404651,3.195349,0.002657,0.0186,2.657143
12,WR,Julio Jones,7100,ATL,19.00,18.0,7100.0,15.068571,2.931429,0.002535,0.0180,2.535211
39,WR,Mike Evans,6300,TB,15.52,15.7,6300.0,12.881270,2.818730,0.002492,0.0157,2.492063


### Tuesday

In [27]:
RB = master_df[master_df['Name'].isin(['Falcons ','Ryan Fitzpatrick',
                'Joe Mixon', 'Kareem Hunt', 'Kenny Golladay', 
                'Terry McLaurin', "Tyler Boyd", 'Eric Ebron', "Mike Davis"])]
WR = master_df[master_df['Name'].isin(['Jets ','Ryan Fitzpatrick',
                'Antonio Gibson', 'Mike Davis', 'Kenny Golladay', 
                'Terry McLaurin', "Tyler Boyd", 'Eric Ebron', "Adam Thielen"])]
TE = master_df[master_df['Name'].isin(['Jets ','Ryan Fitzpatrick',
                'Joe Mixon', 'Mike Davis', 'Kenny Golladay', 
                'Terry McLaurin', "Adam Thielen", 'Eric Ebron', "Jonnu Smith"])]

####  RB=140.6, WR=140.9, TE=139.4

### Thursday

In [28]:
RB = master_df[master_df['Name'].isin(['Jets ','Kyler Murray',
                'Aaron Jones', 'Alvin Kamara', 'Diontae Johnson', 
                'Keenan Allen', "Terry McLaurin", 'Hunter Henry', "Antonio Gibson"])]
WR = master_df[master_df['Name'].isin(['Cardinals ','Kyler Murray',
                'Alvin Kamara', 'Mike Davis', 'Diontae Johnson', 
                'Terry McLaurin', "Tyler Boyd", 'Hunter Henry', "Keenan Allen"])]
TE = master_df[master_df['Name'].isin(['Jets ','Kyler Murray',
                'Alvin Kamara', 'Kareem Hunt', 'Diontae Johnson', 
                'Terry McLaurin', "Tyler Boyd", 'Hunter Henry', "Travis Kelce"])]

#### RB=147.9, WR=148, TE=147.3

### Sunday

In [11]:
RB = master_df[master_df['Name'].isin(['Jets ','Ryan Fitzpatrick',
                'Joe Mixon', 'Antonio Gibson', 'Kenny Golladay', 
                'Terry McLaurin', "Tyler Boyd", 'Jonnu Smith', "Mike Davis"])]
WR = master_df[master_df['Name'].isin(['Vikings ','Ryan Fitzpatrick',
                'Derrick Henry', 'Mike Davis', 'Kenny Golladay', 
                'Terry McLaurin', "Tyler Boyd", 'Irv Smith Jr', "Adam Thielen"])]
TE = master_df[master_df['Name'].isin(['Jets ','Ryan Fitzpatrick',
                'Joe Mixon', 'Mike Davis', 'Kenny Golladay', 
                'Terry McLaurin', "Adam Thielen", 'Eric Ebron', "Jonnu Smith"])]

#### RB=140.6, WR=140.9, TE=139.4

In [29]:
print(RB['Projected Points'].sum(),RB['Salary'].sum())

147.89999999999998 49900


In [30]:
print(WR['Projected Points'].sum(),WR['Salary'].sum())

148.00000000000003 50000


In [31]:
print(TE['Projected Points'].sum(),TE['Salary'].sum())

147.3 50000
